In [2]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF4'

Mounted at /content/drive


In [3]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [4]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [5]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
df_META = df_META.sort_values('date').reset_index(drop=True)

In [6]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
df_AAPL = df_AAPL.sort_values('date').reset_index(drop=True)

In [7]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
df_MSFT = df_MSFT.sort_values('date').reset_index(drop=True)

In [8]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
df_NVDA = df_NVDA.sort_values('date').reset_index(drop=True)

In [9]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])
df_GOOG = df_GOOG.sort_values('date').reset_index(drop=True)

In [10]:
train_size = int(len(df_META) * 0.75)
test_size = len(df_META) - train_size

# META

## Train Test

In [11]:
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [12]:
X_META = train_META.drop(columns=["Stockprice_META","date","sentiment_META","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_META = train_META["Stockprice_META"]

In [13]:
scaler = MinMaxScaler()

X_META = scaler.fit_transform(X_META)

In [14]:
X_META.shape

(564, 10)

In [15]:
model = XGBRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_META , y_META)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_META)
mse = mean_squared_error(y_META, y_pred)
rmse = root_mean_squared_error(y_META, y_pred)
mae = mean_absolute_error(y_META, y_pred)
mape = mean_absolute_percentage_error(y_META, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'alpha': 0, 'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 600, 'reg_lambda': 0}
Test Set Mean Squared Error: 0.1708
Test Set Root Mean Squared Error: 0.4132
Test Set Mean Absolute Error: 0.3262
Test Set Mean Absolute Percentage Error: 0.0014


#AAPL

In [16]:
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [17]:
X_AAPL = train_AAPL.drop(columns=["Stockprice_AAPL","sentiment_AAPL", "date", "Interest", "US_Treasury_Yield", "RETAIL_SALES", "US_Dollar_Index"])
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [18]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [19]:
model = XGBRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_AAPL , y_AAPL)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_AAPL)
mse = mean_squared_error(y_AAPL, y_pred)
rmse = root_mean_squared_error(y_AAPL, y_pred)
mae = mean_absolute_error(y_AAPL, y_pred)
mape = mean_absolute_percentage_error(y_AAPL, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'alpha': 1, 'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 600, 'reg_lambda': 0.1}
Test Set Mean Squared Error: 0.1582
Test Set Root Mean Squared Error: 0.3978
Test Set Mean Absolute Error: 0.3100
Test Set Mean Absolute Percentage Error: 0.0019


#MSFT

In [20]:
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [21]:
X_MSFT = train_MSFT.drop(columns=["Stockprice_MSFT","sentiment_MSFT","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [22]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [23]:
model = XGBRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_MSFT , y_MSFT)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_MSFT)
mse = mean_squared_error(y_MSFT, y_pred)
rmse = root_mean_squared_error(y_MSFT, y_pred)
mae = mean_absolute_error(y_MSFT, y_pred)
mape = mean_absolute_percentage_error(y_MSFT, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'alpha': 0.1, 'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 600, 'reg_lambda': 0.1}
Test Set Mean Squared Error: 0.8225
Test Set Root Mean Squared Error: 0.9069
Test Set Mean Absolute Error: 0.7075
Test Set Mean Absolute Percentage Error: 0.0023


#NVDA

In [24]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [25]:
X_NVDA = train_NVDA.drop(columns=["Stockprice_NVDA","sentiment_NVDA","date", "Interest","RETAIL_SALES", "US_Dollar_Index"])
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [26]:
model = XGBRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_NVDA , y_NVDA)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_NVDA)
mse = mean_squared_error(y_NVDA, y_pred)
rmse = root_mean_squared_error(y_NVDA, y_pred)
mae = mean_absolute_error(y_NVDA, y_pred)
mape = mean_absolute_percentage_error(y_NVDA, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'alpha': 0, 'colsample_bytree': 0.1, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 600, 'reg_lambda': 0}
Test Set Mean Squared Error: 0.4906
Test Set Root Mean Squared Error: 0.7004
Test Set Mean Absolute Error: 0.5346
Test Set Mean Absolute Percentage Error: 0.0187


#GOOG

In [27]:
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [28]:
X_GOOG = train_GOOG.drop(columns=["Stockprice_GOOGL", "sentiment_GOOGL", "date", "Interest", "RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [29]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [30]:
model = XGBRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_GOOG , y_GOOG)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_GOOG)
mse = mean_squared_error(y_GOOG, y_pred)
rmse = root_mean_squared_error(y_GOOG, y_pred)
mae = mean_absolute_error(y_GOOG, y_pred)
mape = mean_absolute_percentage_error(y_GOOG, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

Best Hyperparameters: {'alpha': 1, 'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 200, 'reg_lambda': 0.1}
Test Set Mean Squared Error: 0.8878
Test Set Root Mean Squared Error: 0.9422
Test Set Mean Absolute Error: 0.7313
Test Set Mean Absolute Percentage Error: 0.0061
